In [ ]:
# This notebook can be used to plot our hadronic mass measurements, presented in the appendix.

In [ ]:
import ROOT
import numpy as np
import matplotlib.pyplot as plt
from ipynb.fs.full.CoefficientsCalcPlus import GetCoefficientsFlux
from ipynb.fs.full.CoefficientsCalcPlus import get_normalization
import sklearn.linear_model
import scipy
import warnings; warnings.simplefilter('ignore')
import os
from matplotlib.pyplot import figure
import random as rnd
import matplotlib.ticker as ticker

%jsroot on

# Run these blocks once to prepare histograms file (takes a while!)

In [ ]:
files = ["/eos/home-a/amgruber/SWAN_projects/XSec_ROOT/TTreeMerges/proj"+str(i+1)+"_merged.root" for i in range(58)]
# Change this to your preferred location; Merged trees can be produced using MergeTTrees.ipynb

In [ ]:
coeffs, std = GetCoefficientsFlux(0.75,0.07,1e-12,model=sklearn.linear_model.Ridge,years=1)
coeffs = np.array(coeffs)
norm = get_normalization(coeffs)
years = 20
res = []  # [toy][omega bin]

normalization = open("norms.txt", "r").read().split("\n")[:-1]
normalization = np.array([float(n) for n in normalization])

total_Nbins = int(16000)
bin_min = -int(8000)
bin_max = int(8000)

Q2_reco = "((pmu_4mom.Px()**2 + pmu_4mom.Py()**2 + (750-pmu_4mom.Pz())**2)-(750-ELep)**2)"

nucleon_mass = str((938.272 + 939.565)/2.0)
W_reco = "sqrt("+nucleon_mass+"**2 + 2*"+nucleon_mass+"*(750-ELep)-"+Q2_reco+")"

modes = {"CCQE": "Mode == 1",
         "2p2h": "Mode == 2",
         "RES": "Mode == 11 || Mode == 12 || Mode == 13",
         "Other": "Mode != 1 && Mode != 2 && Mode != 11 && Mode != 12 && Mode != 13"}

reco_hist = ROOT.TH1F("reco", "reco", total_Nbins, bin_min, bin_max)
mode_hists = [ROOT.TH1F(mode, mode, total_Nbins, bin_min, bin_max) for mode in modes]

modes_arrays = [[] for _ in modes]
bins = 58

for i in range(bins):
    print(i)
    file = ROOT.TFile.Open(files[i], "read")
    t = file.Get("FlatTree_VARS")
    
    reco_added = ROOT.TH1F(f"reco_added{i}", "added", total_Nbins, bin_min, bin_max)
    modes_added = [ROOT.TH1F(f"{mode}_added{i}", "added", total_Nbins, bin_min, bin_max) for mode in modes]
    
    t.Project(f"reco_added{i}", W_reco)  # reco
    
    scale_factor = years * normalization[i] / t.GetEntries()
    reco_added.Scale(scale_factor)
    
    for mode, hist in zip(modes.keys(), modes_added):
        t.Project(hist.GetName(), W_reco, modes[mode])
        hist.Scale(scale_factor)
    
    # Properly set bin errors
    for j in range(1, total_Nbins + 1):
        bin_content = reco_added.GetBinContent(j)
        reco_added.SetBinError(j, np.sqrt(bin_content))

    reco_hist.Add(reco_added, coeffs[i])
    
    for mode_hist, mode_added in zip(mode_hists, modes_added):
        mode_hist.Add(mode_added, coeffs[i])
    
    file.Close()

output_file = ROOT.TFile("output_histograms_W.root", "RECREATE")
reco_hist.Write()
for mode_hist in mode_hists:
    mode_hist.Write()
output_file.Close()

# Plot block (after preparation)

In [ ]:
file = ROOT.TFile.Open("output_histograms_W.root", "READ")
if not file or file.IsZombie():
    print(f"Error: Cannot open file {filename}")
    
# Load histograms
reco_hist = file.Get("reco")
if not reco_hist:
    print("Error: reco histogram not found!")

mode_names = ["CCQE", "2p2h", "RES", "Other"]
mode_hists = {mode: file.Get(mode) for mode in mode_names}

for mode, hist in mode_hists.items():
    if not hist:
        print(f"Error: {mode} histogram not found!")


In [ ]:
reco_hist_copy = reco_hist.Clone("reco_hist_copy")
cloned_mode_hists = [hist.Clone(f"{hist.GetName()}_clone") for hist in mode_hists.values()]
reco_hist_copy.Rebin(20)
reco_hist_copy.Scale(1e-4)
for hist in cloned_mode_hists:
    hist.Rebin(20)
    hist.Scale(1e-4)

In [ ]:
# Define color and style mappings with improved styling
styles = {
    "Total": {"color": ROOT.kBlack, "style": ROOT.kDashed, "width": 3},  # Thicker black dashed line
    "CCQE": {"color": ROOT.kBlue, "style": ROOT.kDashed, "width": 3},
    "2p2h": {"color": ROOT.kOrange+7, "style": ROOT.kDashed, "width": 3},
    "RES": {"color": ROOT.kGreen+2, "style": ROOT.kDashed, "width": 3},
    "Other": {"color": ROOT.kRed, "style": ROOT.kDashed, "width": 3},
    "Reco": {"color": ROOT.kBlack, "marker_style": 20, "marker_size": 1.0},
}

# Apply styles to mode histograms
for mode, hist in zip(mode_names, cloned_mode_hists):
    style = styles.get(mode, {})  # Get style dictionary for mode
    hist.SetLineColor(style.get("color", ROOT.kBlack))
    hist.SetLineStyle(style.get("style", ROOT.kSolid))
    hist.SetLineWidth(style.get("width", 3))  # Thicker lines
    hist.SetStats(False)
    hist.SetTitle("")

# Create a "Total" histogram as the sum of all modes
total_hist = cloned_mode_hists[0].Clone("Total")
total_hist.Reset()
for hist in cloned_mode_hists:
    total_hist.Add(hist)

total_hist.SetLineColor(ROOT.kBlack)
total_hist.SetLineStyle(ROOT.kDashed)
total_hist.SetLineWidth(3)
total_hist.SetStats(False)
total_hist.SetTitle("")

# Apply reco histogram styling
reco_hist_copy.SetMarkerStyle(20)
reco_hist_copy.SetMarkerSize(1.0)
reco_hist_copy.SetMarkerColor(ROOT.kBlack)
reco_hist_copy.SetLineColor(ROOT.kBlack)
reco_hist_copy.SetLineWidth(3)
reco_hist_copy.SetStats(False)
reco_hist_copy.SetTitle("")

# Set axis labels
reco_hist_copy.SetXTitle("W_{reco} [MeV/c^{2}]")
reco_hist_copy.SetYTitle("Events [a.u.]")
total_hist.SetXTitle("W [MeV/c^{2}]")
total_hist.SetYTitle("Events [arbitrary units]")

# Create legend with better stat uncertainty representation
legend = ROOT.TLegend(0.6, 0.55, 0.9, 0.9)
legend.SetTextSize(0.08)
legend.SetBorderSize(0)
legend.SetFillStyle(0)
legend.AddEntry(reco_hist_copy, "Stat Unc", "lep")  # "lep" for points with error bars
legend.AddEntry(total_hist, "Total", "l")
for mode, hist in zip(mode_names, cloned_mode_hists):
    legend.AddEntry(hist, mode, "l")

# Create and configure canvas
c = ROOT.TCanvas("c", "Comparison Canvas", 2400, 1600)
c.SetLeftMargin(0.15)
c.SetGrid()

# Draw histograms
total_hist.Draw("hist")
reco_hist_copy.Draw("e1 same")
for hist in cloned_mode_hists:
    hist.Draw("hist same")

# Adjust X-axis range
max_bin = 2200
reco_hist_copy.GetXaxis().SetRangeUser(0, max_bin)
reco_hist_copy.GetYaxis().SetRangeUser(-0.3, 5.7)
total_hist.GetXaxis().SetRangeUser(0, max_bin)
total_hist.GetYaxis().SetRangeUser(-0.3, 5.7)
for hist in cloned_mode_hists:
    hist.GetXaxis().SetRangeUser(0, max_bin)
    hist.GetYaxis().SetRangeUser(-0.3, 5.7)

# Draw legend
legend.Draw()

# Finalize canvas
c.SetTitle("")
c.Draw()